In [1]:
!pip install --upgrade pip
!pip install ortools
from pathlib import Path
import math, pandas as pd
from collections import defaultdict
from ortools.sat.python import cp_model
import pandas as pd
import math
import csv

     ---------------------------------------- 1.8/1.8 MB 10.1 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\Tanmay\anaconda3\python.exe -m pip install --upgrade pip


In [2]:
import sys
!"{sys.executable}" -m pip install ortools

In [ ]:
# JUPYTER CELL — NBA-style scheduler (CP-SAT, no pandas/numpy)
# - Soft 3-in-4 and soft fixed-days (penalties) for feasibility
# - Exact host-to-host travel in the objective
# - Outputs: solve_report.txt and Scheduled_CP_SAT_ExactTravel.csv

# =========================
# Paths (double forward slashes, per your request)
# =========================
BASE = Path("C://Users//Tanmay//Downloads//UNIVERSITY OF MELBOURNE STUDY FOLDER//Scheduling and Optimisation")
TEAMS_CSV = BASE / "Teams_NBA.csv"
GAMES_CSV = BASE / "Games_NBA.csv"  # optional; if missing we auto-generate

# =========================
# Config knobs (tune here)
# =========================
DAYS_HORIZON   = 180       # must be >= any fixed day, increase if tight
W_MAKESPAN     = 1.0       # weight for last-day Cmax
W_BREAKS       = 0.05      # weight for idle-day gaps
W_TRAVEL       = 0.01      # weight for host-to-host travel (km)

SOFT_3IN4      = True      # penalize >3 in any 4-day window instead of forbidding
SOFT_3IN4_W    = 1000      # penalty per excess game in a window

SOFT_FIXED     = True      # penalize moving a fixed-day game instead of forbidding
SOFT_FIXED_W   = 5000      # penalty per fixed-day violation

TIME_LIMIT_SEC = 900       # solver time budget
NUM_WORKERS    = 8         # threads

# =========================
# CSV helpers (no pandas)
# =========================
def read_csv_lower(path: Path):
    rows = []
    with path.open("r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        if rdr.fieldnames is None:
            return rows
        for raw in rdr:
            rows.append({(k or "").strip().lower(): (v.strip() if isinstance(v, str) else v)
                         for k, v in raw.items()})
    return rows

def pick_col(cols, candidates, required=False):
    for c in candidates:
        if c in cols: return c
    if required:
        raise ValueError(f"Missing required column; expected one of {candidates}")
    return None

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# =========================
# Loaders (teams, games)
# =========================
def load_league(teams_csv: Path):
    rows = read_csv_lower(teams_csv)
    if not rows:
        raise ValueError("Teams CSV is empty.")

    cols = rows[0].keys()
    team_col  = pick_col(cols, ["team","abbr","code","name"], required=True)
    venue_col = pick_col(cols, ["venue","home_venue","arena","city","home"])
    conf_col  = pick_col(cols, ["conference","conf"])
    lat_col   = pick_col(cols, ["lat","latitude"])
    lon_col   = pick_col(cols, ["lon","lng","longitude"])

    team_list = [str(r[team_col]).upper() for r in rows]
    # home venues
    if venue_col:
        home_venues = {str(r[team_col]).upper(): r[venue_col] for r in rows}
    else:
        home_venues = {t: f"{t}_HOME" for t in team_list}
    # conferences (unused but kept for completeness)
    if conf_col:
        conf = {str(r[team_col]).upper(): str(r[conf_col])[0].upper() for r in rows}
    else:
        conf = {t: "E" for t in team_list}

    # distances (try lat/lon; else fallback 100 km off-diagonal)
    have_coords = (lat_col and lon_col)
    coords = {}
    if have_coords:
        try:
            for r in rows:
                t = str(r[team_col]).upper()
                coords[t] = (float(r[lat_col]), float(r[lon_col]))
        except Exception:
            have_coords = False

    if have_coords:
        distance = defaultdict(dict)
        for t1 in team_list:
            for t2 in team_list:
                distance[t1][t2] = 0 if t1 == t2 else int(round(haversine_km(*coords[t1], *coords[t2])))
    else:
        distance = defaultdict(lambda: defaultdict(int))
        for t1 in team_list:
            for t2 in team_list:
                distance[t1][t2] = 0 if t1 == t2 else 100

    return team_list, home_venues, conf, distance

def load_games_optional(games_csv: Path, team_list, home_venues):
    """
    Returns:
      games: list {id, home, away, venue}
      fixed_days: {game_id: day}
    """
    if not games_csv.exists():
        gid, games = 0, []
        for h in team_list:
            for a in team_list:
                if a != h:
                    games.append({"id": gid, "home": h, "away": a, "venue": home_venues[h]})
                    gid += 1
        return games, {}

    rows = read_csv_lower(games_csv)
    if not rows:
        raise ValueError("Games CSV is empty.")

    cols = rows[0].keys()
    home_col = pick_col(cols, ["home","home_team","host"], required=True)
    away_col = pick_col(cols, ["away","away_team","visitor"], required=True)
    venue_col = pick_col(cols, ["venue","arena","home_venue"])
    day_col   = pick_col(cols, ["day","date","day_index","slot"])

    games, fixed = [], {}
    gid = 0
    for r in rows:
        h = str(r[home_col]).upper()
        a = str(r[away_col]).upper()
        if h not in team_list or a not in team_list:
            raise ValueError(f"Unknown team in games row: {h} vs {a}")
        venue = r[venue_col] if venue_col else home_venues[h]
        games.append({"id": gid, "home": h, "away": a, "venue": venue})
        if day_col and r.get(day_col, "") != "":
            try:
                fixed[gid] = int(float(r[day_col]))  # tolerate "12.0"
            except Exception:
                pass
        gid += 1
    return games, fixed

# =========================
# Sanity & seeding
# =========================
def sanity_check(team_list, games, FIXED, days_horizon):
    from collections import Counter
    team_day = defaultdict(lambda: defaultdict(int))
    venue_day = defaultdict(lambda: defaultdict(int))
    for g in games:
        gid = g["id"]
        if gid in FIXED:
            d = FIXED[gid]
            team_day[g["home"]][d] += 1
            team_day[g["away"]][d] += 1
            venue_day[g["venue"]][d] += 1
    bad_team = [(t,d,c) for t,dd in team_day.items() for d,c in dd.items() if c>1]
    bad_venue = [(v,d,c) for v,dd in venue_day.items() for d,c in dd.items() if c>1]
    if bad_team:
        print("⚠️ Fixed-day TEAM clashes:", bad_team)
    if bad_venue:
        print("⚠️ Fixed-day VENUE clashes:", bad_venue)

    home_counts = Counter(g["home"] for g in games)
    away_counts = Counter(g["away"] for g in games)
    games_per_team = {t: home_counts.get(t,0)+away_counts.get(t,0) for t in team_list}
    lb_team = max(games_per_team.values()) if games_per_team else 0
    lb_3in4 = max(int(math.ceil(4/3 * g)) for g in games_per_team.values()) if games_per_team else 0
    venue_counts = {v: sum(1 for g in games if g["venue"]==v) for v in {g["venue"] for g in games}}
    lb_venue = max(venue_counts.values()) if venue_counts else 0
    LB = max(lb_team, lb_3in4, lb_venue)
    print(f"Sanity lower bounds → team-days: {lb_team}, 3-in-4: {lb_3in4}, venue: {lb_venue}.  Horizon={days_horizon}.")
    if days_horizon < LB:
        print("👉 Horizon < lower bound: increase DAYS_HORIZON.")

def greedy_seed_days(team_list, games, FIXED, days):
    schedule = defaultdict(list)
    team_day_used = defaultdict(lambda: defaultdict(bool))
    venue_day_used = defaultdict(lambda: defaultdict(bool))
    seed_day = {}
    # fixed first
    for g in games:
        gid = g["id"]
        if gid in FIXED:
            d = FIXED[gid]
            schedule[d].append(gid)
            seed_day[gid] = d
            team_day_used[g["home"]][d] = True
            team_day_used[g["away"]][d] = True
            venue_day_used[g["venue"]][d] = True
    # then others
    for g in games:
        gid = g["id"]
        if gid in seed_day: continue
        for d in days:
            if not team_day_used[g["home"]][d] and not team_day_used[g["away"]][d] and not venue_day_used[g["venue"]][d]:
                schedule[d].append(gid)
                seed_day[gid] = d
                team_day_used[g["home"]][d] = True
                team_day_used[g["away"]][d] = True
                venue_day_used[g["venue"]][d] = True
                break
    return seed_day

# =========================
# Build & solve model
# =========================
def solve_schedule():
    # Load inputs
    team_list, home_venues, conf, distance = load_league(TEAMS_CSV)
    games, FIXED = load_games_optional(GAMES_CSV, team_list, home_venues)

    # Horizon and sets
    max_fixed = max(FIXED.values()) if FIXED else 0
    H = max(DAYS_HORIZON, max_fixed, 1)
    D = list(range(1, H+1))

    # Helper indexes
    games_of_team = {t: [] for t in team_list}
    g_home = {}
    for g in games:
        gid = g["id"]
        g_home[gid] = g["home"]
        games_of_team[g["home"]].append(gid)
        games_of_team[g["away"]].append(gid)
    venues = list({g["venue"] for g in games})

    # Logs
    sanity_check(team_list, games, FIXED, H)

    # Model
    m = cp_model.CpModel()

    # Decision: x[g,d]
    x = {(g["id"], d): m.NewBoolVar(f"x_g{g['id']}_d{d}") for g in games for d in D}

    # Fixed vs soft-fixed and "each game exactly once"
    fix_penalties = []
    if SOFT_FIXED:
        for g in games:
            gid = g["id"]
            m.Add(sum(x[(gid, d)] for d in D) == 1)
            if gid in FIXED:
                fd = FIXED[gid]
                v = m.NewBoolVar(f"fixed_ok_g{gid}")
                m.Add(v == x[(gid, fd)])
                fix_penalties.append(1 - v)  # penalize if not on fixed day
    else:
        for g in games:
            gid = g["id"]
            if gid in FIXED:
                fd = FIXED[gid]
                for d in D:
                    m.Add(x[(gid, d)] == (1 if d == fd else 0))
            else:
                m.Add(sum(x[(gid, d)] for d in D) == 1)

    # Team capacity & Venue capacity
    for t in team_list:
        for d in D:
            m.Add(sum(x[(gid, d)] for gid in games_of_team[t]) <= 1)
    for v in venues:
        for d in D:
            m.Add(sum(x[(g["id"], d)] for g in games if g["venue"] == v) <= 1)

    # Day of each game: D_g
    D_g = {}
    for g in games:
        gid = g["id"]
        var = m.NewIntVar(min(D), max(D), f"D_g{gid}")
        m.Add(var == sum(d * x[(gid, d)] for d in D))
        D_g[gid] = var

    # Per-team round order: z[t,g,r], S[t,r]
    z, S = {}, {}
    for t in team_list:
        T = games_of_team[t]
        k = len(T)
        if k == 0: continue
        for gid in T:
            for r in range(1, k+1):
                z[(t, gid, r)] = m.NewBoolVar(f"z_{t}_g{gid}_r{r}")
        for gid in T:
            m.Add(sum(z[(t, gid, r)] for r in range(1, k+1)) == 1)
        for r in range(1, k+1):
            m.Add(sum(z[(t, gid, r)] for gid in T) == 1)
            S[(t, r)] = m.NewIntVar(min(D), max(D), f"S_{t}_{r}")
            for gid in T:
                m.Add(S[(t, r)] == D_g[gid]).OnlyEnforceIf(z[(t, gid, r)])

    # 3-in-4 (soft or hard)
    soft_3in4_penalties = []
    for t in team_list:
        k = len(games_of_team[t])
        if k == 0: continue
        for w in D[:-3]:
            inwin = []
            for r in range(1, k+1):
                b_ge = m.NewBoolVar(f"ge_{t}_r{r}_w{w}")
                b_le = m.NewBoolVar(f"le_{t}_r{r}_w{w}")
                m.Add(S[(t, r)] >= w).OnlyEnforceIf(b_ge)
                m.Add(S[(t, r)] <= w - 1).OnlyEnforceIf(b_ge.Not())
                m.Add(S[(t, r)] <= w + 3).OnlyEnforceIf(b_le)
                m.Add(S[(t, r)] >= w + 4).OnlyEnforceIf(b_le.Not())
                a = m.NewBoolVar(f"inwin_{t}_r{r}_w{w}")
                m.Add(a <= b_ge); m.Add(a <= b_le); m.Add(a >= b_ge + b_le - 1)
                inwin.append(a)
            if SOFT_3IN4:
                over = m.NewIntVar(0, 4, f"over3in4_{t}_w{w}")
                m.Add(over >= sum(inwin) - 3)
                soft_3in4_penalties.append(over)
            else:
                m.Add(sum(inwin) <= 3)

    # Breaks between rounds
    break_vars = []
    for t in team_list:
        k = len(games_of_team[t])
        for r in range(1, k):
            b = m.NewIntVar(0, len(D), f"break_{t}_{r}")
            m.Add(b >= S[(t, r+1)] - S[(t, r)] - 1)
            break_vars.append(b)

    # Makespan
    Cmax = m.NewIntVar(min(D), max(D), "Cmax")
    for g in games:
        m.Add(Cmax >= D_g[g["id"]])

    # Exact travel chaining (hosts)
    u, w_and, travel_terms = {}, {}, []
    games_by_host_for_team = {t: defaultdict(list) for t in team_list}
    for t in team_list:
        for gid in games_of_team[t]:
            games_by_host_for_team[t][g_home[gid]].append(gid)
    for t in team_list:
        k = len(games_of_team[t])
        if k == 0: continue
        for r in range(1, k+1):
            for h in team_list:
                u[(t, r, h)] = m.NewBoolVar(f"u_{t}_r{r}_host{h}")
            for h in team_list:
                gids = games_by_host_for_team[t].get(h, [])
                if len(gids) == 0:
                    m.Add(u[(t, r, h)] == 0)
                else:
                    m.Add(u[(t, r, h)] == sum(z[(t, gid, r)] for gid in gids))
            m.Add(sum(u[(t, r, h)] for h in team_list) == 1)
        for r in range(1, k):
            for h1 in team_list:
                for h2 in team_list:
                    key = (t, r, h1, h2)
                    w_and[key] = m.NewBoolVar(f"w_{t}_r{r}_{h1}_{h2}")
                    m.Add(w_and[key] <= u[(t, r, h1)])
                    m.Add(w_and[key] <= u[(t, r+1, h2)])
                    m.Add(w_and[key] >= u[(t, r, h1)] + u[(t, r+1, h2)] - 1)
                    dist = int(distance[h1][h2])
                    if dist:
                        travel_terms.append((w_and[key], dist))

    # Objective (integerized weights)
    WM = int(round(W_MAKESPAN * 1000))
    WB = int(round(W_BREAKS   * 1000))
    WT = int(round(W_TRAVEL   * 1))
    terms = [WM * Cmax, WB * sum(break_vars), WT * sum(dist * w for (w, dist) in travel_terms)]
    if SOFT_3IN4 and soft_3in4_penalties:
        terms.append(int(SOFT_3IN4_W) * sum(soft_3in4_penalties))
    if SOFT_FIXED and fix_penalties:
        terms.append(int(SOFT_FIXED_W) * sum(fix_penalties))
    m.Minimize(sum(terms))

    # Hints: quick greedy seed
    seed = greedy_seed_days(team_list, games, FIXED, D)
    for gid, day in seed.items():
        if day in D:
            m.AddHint(x[(gid, day)], 1)

    # Solve
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = float(TIME_LIMIT_SEC)
    solver.parameters.num_search_workers  = int(NUM_WORKERS)
    solver.parameters.linearization_level = 2
    solver.parameters.log_to_stdout = True
    solver.parameters.log_search_progress = True

    status = solver.Solve(m)
    status_name = solver.StatusName(status)
    report_txt = (
        f"Status: {status_name}\n"
        f"Wall time (s): {solver.WallTime():.2f}\n"
        f"Objective (if any): {getattr(solver, 'ObjectiveValue', lambda: float('nan'))()}\n"
        f"Best bound: {getattr(solver, 'BestObjectiveBound', lambda: float('nan'))()}\n"
        f"Branches: {getattr(solver, 'NumBranches', lambda: 0)()}\n"
        f"Conflicts: {getattr(solver, 'NumConflicts', lambda: 0)()}\n"
        f"Response stats:\n{solver.ResponseStats()}\n"
    )
    (BASE / "solve_report.txt").write_text(report_txt, encoding="utf-8")
    print(report_txt)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        Cmax_val = solver.Value(Cmax)
        total_breaks = sum(solver.Value(b) for b in break_vars)
        travel_km = 0
        for (t, r, h1, h2), var in w_and.items():
            if solver.Value(var) == 1:
                travel_km += int(distance[h1][h2])

        # Soft-constraint summaries
        if SOFT_3IN4:
            realized_over_3in4 = sum(int(solver.Value(v)) for v in soft_3in4_penalties)
            print(f"3-in-4 violations (sum over windows): {realized_over_3in4}")
        if SOFT_FIXED:
            moved_fixed = sum(int(solver.Value(p)) for p in fix_penalties)
            print(f"Fixed-day moves (games not on specified day): {moved_fixed}")

        print(f"Cmax (last day): {Cmax_val}")
        print(f"Breaks total: {total_breaks}")
        print(f"Travel (km): {travel_km}")

        # Write CSV
        assign_day = {g["id"]: None for g in games}
        for g in games:
            gid = g["id"]
            for d in D:
                if solver.Value(x[(gid, d)]) == 1:
                    assign_day[gid] = d
                    break

        out_csv = BASE / "Scheduled_CP_SAT_ExactTravel.csv"
        with out_csv.open("w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["day","away","home","venue","game_id"])
            for g in sorted(games, key=lambda G: (assign_day[G["id"]] or 10**9, G["id"])):
                w.writerow([assign_day[g["id"]], g["away"], g["home"], g["venue"], g["id"]])
        print(f"✅ Saved → {out_csv}")
    else:
        print("❌ No solution returned. Check solve_report.txt, raise DAYS_HORIZON, or increase penalties.")

# Run now
solve_schedule()


Sanity lower bounds → team-days: 82, 3-in-4: 110, venue: 82.  Horizon=180.
